In [1]:
import pandas as pd
import requests
import csv
import os
import zipfile
import io
import glob

## fyers.in

In [2]:

url_fyers = "https://public.fyers.in/sym_details/NSE_FO.csv"
df_fyers = pd.read_csv(url_fyers)
df_fyers.columns

Index(['101124102835239', 'MIDCPNIFTY 24 Oct 28 FUT', '11', '50', '0.05',
       'Unnamed: 5', '0915-1530|1815-1915:', '2024-10-25', '1730109600',
       'NSE:MIDCPNIFTY24OCTFUT', '10', '11.1', '35239', 'MIDCPNIFTY', '26074',
       '-1.0', 'XX', '101000000026074', 'None', '0', '0.0'],
      dtype='object')

In [3]:
new_column_names = [
        'Instrument_ID', 'Instrument_Name', 'Segment_ID', 'Lot_Size', 
        'Tick_Size', 'Unnamed_Column', 'Trading_Hours', 'Expiry_Date', 
        'Timestamp', 'Exchange_Symbol', 'Precision', 'Multiplier', 
        'Unique_Code', 'Underlying_Asset', 'Asset_Code', 'Price_Change',
        'Settlement_Type', 'Trading_Code', 'Option_Type', 
        'Strike_Price', 'Open_Interest'
    ]
df_fyers.columns = new_column_names

In [4]:
df_fyers['Expiry_Date'] = pd.to_datetime(df_fyers['Expiry_Date'], errors='coerce')

In [5]:
df_fyers.head()

,Instrument_ID,Instrument_Name,Segment_ID,Lot_Size,Tick_Size,Unnamed_Column,Trading_Hours,Expiry_Date,Timestamp,Exchange_Symbol,...,Multiplier,Unique_Code,Underlying_Asset,Asset_Code,Price_Change,Settlement_Type,Trading_Code,Option_Type,Strike_Price,Open_Interest
0,101124102835513,MIDCPNIFTY 24 Oct 28 15350 CE,14,50,0.05,NaN,0915-1530|1815-1915:,2024-10-25,1730109600,NSE:MIDCPNIFTY24OCT15350CE,...,11,35513,MIDCPNIFTY,26074,15350.0,CE,101000000026074,None,0,0.0
1,101124102835515,MIDCPNIFTY 24 Oct 28 15350 PE,14,50,0.05,NaN,0915-1530|1815-1915:,2024-10-25,1730109600,NSE:MIDCPNIFTY24OCT15350PE,...,11,35515,MIDCPNIFTY,26074,15350.0,PE,101000000026074,None,0,0.0
2,101124102835516,MIDCPNIFTY 24 Oct 28 15375 CE,14,50,0.05,NaN,0915-1530|1815-1915:,2024-10-25,1730109600,NSE:MIDCPNIFTY24OCT15375CE,...,11,35516,MIDCPNIFTY,26074,15375.0,CE,101000000026074,None,0,0.0
3,101124102835517,MIDCPNIFTY 24 Oct 28 15375 PE,14,50,0.05,NaN,0915-1530|1815-1915:,2024-10-25,1730109600,NSE:MIDCPNIFTY24OCT15375PE,...,11,35517,MIDCPNIFTY,26074,15375.0,PE,101000000026074,None,0,0.0
4,101124102835518,MIDCPNIFTY 24 Oct 28 17700 CE,14,50,0.05,NaN,0915-1530|1815-1915:,2024-10-25,1730109600,NSE:MIDCPNIFTY24OCT17700CE,...,11,35518,MIDCPNIFTY,26074,17700.0,CE,101000000026074,None,0,0.0


In [6]:
instrument_split = df_fyers['Instrument_Name'].str.split(' ', expand=True)

df_fyers['Underlying_Asset'] = instrument_split[0]
df_fyers['Expiry_Day'] = instrument_split[1]
df_fyers['Expiry_Month'] = instrument_split[2]
df_fyers['Expiry_Year'] = instrument_split[3]
df_fyers['Strike_Price'] = pd.to_numeric(instrument_split[4], errors='coerce')
df_fyers['Option_Type'] = instrument_split[5]

df_fyers['Expiry_Date'] = pd.to_datetime(
        df_fyers['Expiry_Day'] + ' ' + df_fyers['Expiry_Month'] + ' ' + df_fyers['Expiry_Year'],
        errors='coerce'
    )

print(df_fyers.head())


     Instrument_ID                Instrument_Name  Segment_ID  Lot_Size  \
0  101124102835513  MIDCPNIFTY 24 Oct 28 15350 CE          14        50   
1  101124102835515  MIDCPNIFTY 24 Oct 28 15350 PE          14        50   
2  101124102835516  MIDCPNIFTY 24 Oct 28 15375 CE          14        50   
3  101124102835517  MIDCPNIFTY 24 Oct 28 15375 PE          14        50   
4  101124102835518  MIDCPNIFTY 24 Oct 28 17700 CE          14        50   

   Tick_Size  Unnamed_Column         Trading_Hours Expiry_Date   Timestamp  \
0       0.05             NaN  0915-1530|1815-1915:  2028-10-24  1730109600   
1       0.05             NaN  0915-1530|1815-1915:  2028-10-24  1730109600   
2       0.05             NaN  0915-1530|1815-1915:  2028-10-24  1730109600   
3       0.05             NaN  0915-1530|1815-1915:  2028-10-24  1730109600   
4       0.05             NaN  0915-1530|1815-1915:  2028-10-24  1730109600   

              Exchange_Symbol  ...  Asset_Code  Price_Change  Settlement_Type  \

In [7]:
df_fyers.columns

Index(['Instrument_ID', 'Instrument_Name', 'Segment_ID', 'Lot_Size',
       'Tick_Size', 'Unnamed_Column', 'Trading_Hours', 'Expiry_Date',
       'Timestamp', 'Exchange_Symbol', 'Precision', 'Multiplier',
       'Unique_Code', 'Underlying_Asset', 'Asset_Code', 'Price_Change',
       'Settlement_Type', 'Trading_Code', 'Option_Type', 'Strike_Price',
       'Open_Interest', 'Expiry_Day', 'Expiry_Month', 'Expiry_Year'],
      dtype='object')

In [8]:
columns_to_drop = [
        'Unnamed_Column', 'Timestamp', 'Precision', 'Multiplier', 'Trading_Code', 
        'Asset_Code', 'Price_Change', 'Settlement_Type'
    ]

df_fyers = df_fyers.drop(columns=columns_to_drop)
df_fyers.head()

,Instrument_ID,Instrument_Name,Segment_ID,Lot_Size,Tick_Size,Trading_Hours,Expiry_Date,Exchange_Symbol,Unique_Code,Underlying_Asset,Option_Type,Strike_Price,Open_Interest,Expiry_Day,Expiry_Month,Expiry_Year
0,101124102835513,MIDCPNIFTY 24 Oct 28 15350 CE,14,50,0.05,0915-1530|1815-1915:,2028-10-24,NSE:MIDCPNIFTY24OCT15350CE,35513,MIDCPNIFTY,CE,15350.0,0.0,24,Oct,28
1,101124102835515,MIDCPNIFTY 24 Oct 28 15350 PE,14,50,0.05,0915-1530|1815-1915:,2028-10-24,NSE:MIDCPNIFTY24OCT15350PE,35515,MIDCPNIFTY,PE,15350.0,0.0,24,Oct,28
2,101124102835516,MIDCPNIFTY 24 Oct 28 15375 CE,14,50,0.05,0915-1530|1815-1915:,2028-10-24,NSE:MIDCPNIFTY24OCT15375CE,35516,MIDCPNIFTY,CE,15375.0,0.0,24,Oct,28
3,101124102835517,MIDCPNIFTY 24 Oct 28 15375 PE,14,50,0.05,0915-1530|1815-1915:,2028-10-24,NSE:MIDCPNIFTY24OCT15375PE,35517,MIDCPNIFTY,PE,15375.0,0.0,24,Oct,28
4,101124102835518,MIDCPNIFTY 24 Oct 28 17700 CE,14,50,0.05,0915-1530|1815-1915:,2028-10-24,NSE:MIDCPNIFTY24OCT17700CE,35518,MIDCPNIFTY,CE,17700.0,0.0,24,Oct,28


In [9]:
df_fyers.columns

Index(['Instrument_ID', 'Instrument_Name', 'Segment_ID', 'Lot_Size',
       'Tick_Size', 'Trading_Hours', 'Expiry_Date', 'Exchange_Symbol',
       'Unique_Code', 'Underlying_Asset', 'Option_Type', 'Strike_Price',
       'Open_Interest', 'Expiry_Day', 'Expiry_Month', 'Expiry_Year'],
      dtype='object')

In [10]:
df_fyers.to_csv('fyers.csv', index=False)

## Angel One

In [11]:
url_angel_one = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
response = requests.get(url_angel_one)
response.raise_for_status()  

In [12]:
data_angel_one  = response.json()
df_angel_one  = pd.DataFrame(data_angel_one )
df_angel_one

,token,symbol,name,expiry,strike,lotsize,instrumenttype,exch_seg,tick_size
0,6293,GULPOLY-BL,GULPOLY,,-1.000000,1,,NSE,1.000000
1,24051,739UK30-SG,739UK30,,-1.000000,100,,NSE,1.000000
2,3475,THERMAX-EQ,THERMAX,,-1.000000,1,,NSE,5.000000
3,3523,69MZ33-SG,69MZ33,,-1.000000,100,,NSE,1.000000
4,17996,PGIMCSR2G-MF,PGIMCSR2G,,-1.000000,1,,NSE,1.000000
...,...,...,...,...,...,...,...,...,...
117705,126908,GOLD25MAR78900PE,GOLD,28MAR2025,789000000.000000,1,OPTFUT,NCO,5000.000000
117706,126383,SILVERM25JUN105500PE,SILVERM,23JUN2025,1055000000.000000,1,OPTFUT,NCO,5000.000000
117707,126515,SILVERM25JUN84500PE,SILVERM,23JUN2025,845000000.000000,1,OPTFUT,NCO,5000.000000
117708,126997,SILVERM25FEB94000CE,SILVERM,21FEB2025,940000000.000000,1,OPTFUT,NCO,5000.000000


In [13]:
df_angel_one.to_csv('angel_one.csv', index=False)

## icici 

In [14]:
icici_url = "https://directlink.icicidirect.com/NewSecurityMaster/SecurityMaster.zip"
os.makedirs('icici', exist_ok=True)

In [15]:
def extract_zip_and_convert(url, extract_to='icici'):
    response = requests.get(url)
    response.raise_for_status()  

    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(path=extract_to) 
        for filename in z.namelist():
            print(filename)

    for filename in z.namelist():
        if filename.endswith('.txt'):
            with open(os.path.join(extract_to, filename), 'r') as file:
                csv_content = file.read().replace('|', ',')                
            new_filename = os.path.splitext(filename)[0] + '.csv'
            with open(os.path.join(extract_to, new_filename), 'w') as csv_file:
                csv_file.write(csv_content)

            print(f"Converted {filename} to {new_filename}")


In [16]:
extract_zip_and_convert(icici_url)
for file in os.listdir('icici'):
    print(file)


csv_files = [f for f in os.listdir('icici') if f.endswith('.csv')]
if csv_files:
    df_icici = pd.read_csv(os.path.join('icici', csv_files[0]), on_bad_lines='skip')
    print(f"\nData shape: {df_icici.shape}")
    print(df_icici.head())
    
    df_icici.to_csv('icici/icici_master_scrip.csv', index=False)
    print("ICICI data saved as 'icici_master_scrip.csv'.")
else:
    print("No CSV files found in the 'icici' directory.")


BSEScripMaster.txt
CDNSEScripMaster.txt
FOBSEScripMaster.txt
FONSEScripMaster.txt
NSEScripMaster.txt
Converted BSEScripMaster.txt to BSEScripMaster.csv
Converted CDNSEScripMaster.txt to CDNSEScripMaster.csv
Converted FOBSEScripMaster.txt to FOBSEScripMaster.csv
Converted FONSEScripMaster.txt to FONSEScripMaster.csv
Converted NSEScripMaster.txt to NSEScripMaster.csv
BSEScripMaster.csv
BSEScripMaster.txt
CDNSEScripMaster.csv
CDNSEScripMaster.txt
FOBSEScripMaster.csv
FOBSEScripMaster.txt
FONSEScripMaster.csv
FONSEScripMaster.txt
icici_master_scrip.csv
NSEScripMaster.csv
NSEScripMaster.txt

Data shape: (11321, 34)
    Token ShortName Series                 CompanyName  TickSize  LotSize  \
0  800078    03D001     DR  11.10% GOI LOAN 07/04/2003    115.10        1   
1  800004    03E001     DR         GOI LOAN 5.75% 2003    110.20        1   
2  800032    03E002     DR           GOI LOAN 11% 2003    112.25        1   
3  800049    04C001     DR  12.50% GOI LOAN 23/03/2004    114.40        1 

In [17]:
path = 'icici/'
csv_files = glob.glob(path + "*.csv")

In [18]:
file_paths = csv_files
columns_to_keep = [
    'Token', 'ShortName', 'Series', 'CompanyName', 'TickSize', 'LotSize',
    'ScripCode', 'MarketLot', 'BCastFlag', 'AVMBuyMargin', 'AVMSellMargin', 
    'ScripID', 'ScripName', 'GroupName', 'NdFlag', 'NDSDate', 'NDEDate',
    'SuspStatus', 'avmflag', 'SuspensionReason', 'Suspensiondate', 
    'DateOfListing', 'DateOfDeListing', 'IssuePrice', 'FaceValue', 'ISINCode',
    '52WeeksHigh', '52WeeksLow', 'LifeTimeHigh', 'LifeTimeLow', 'HighDate', 
    'LowDate', 'MarginPercentage', 'ExchangeCode', 'SourceFile', 
    'InstrumentName', 'ExpiryDate', 'StrikePrice', 'OptionType', 'CALevel', 
    'PermittedToTrade', 'IssueCapital', 'WarningQty', 'FreezeQty', 'CreditRating', 
    'NormalMarketStatus', 'OddLotMarketStatus', 'SpotMarketStatus', 
    'AuctionMarketStatus', 'NormalMarketEligibility', 'OddLotMarketEligibility', 
    'SpotMarketEligibility', 'AuctionMarketEligibility', 'IssueRate', 'IssueStartDate', 
    'InterestPaymentDate', 'IssueMaturityDate', 'MinimumLotQty', 'ListingDate', 
    'ExpulsionDate', 'ReAdmissionDate', 'RecordDate', 'LowPriceRange', 'HighPriceRange', 
    'SecurityExpiryDate', 'NoDeliveryStartDate', 'NoDeliveryEndDate', 'MF', 'AON', 
    'ParticipantInMarketIndex', 'BookClsStartDate', 'BookClsEndDate', 
    'ExcerciseStartDate', 'ExcerciseEndDate', 'OldToken', 'AssetInstrument', 
    'AssetName', 'AssetToken', 'IntrinsicValue', 'ExtrinsicValue', 'ExcerciseStyle', 
    'EGM', 'AGM', 'Interest', 'Bonus', 'Rights', 'Dividends', 'ExAllowed', 
    'ExRejectionAllowed', 'PlAllowed', 'IsThisAsset', 'IsCorpAdjusted', 
    'LocalUpdateDatetime', 'DeleteFlag', 'Remarks', 'BasePrice'
]



In [19]:
all_data = []
for file in file_paths:
    try:
        df = pd.read_csv(file)
        df['SourceFile'] = file 
        missing_columns = [col for col in columns_to_keep if col not in df.columns]
        for col in missing_columns:
            df[col] = pd.NA 
        df = df[columns_to_keep] 
        all_data.append(df)
    except Exception as e:
        print(f"Error processing {file}: {e}")

In [20]:
merged_data_icici = pd.concat(all_data, ignore_index=True).drop_duplicates(subset=['Token'])

In [21]:
merged_data_icici.fillna('', inplace=True)

In [22]:
merged_data_icici.columns

Index(['Token', 'ShortName', 'Series', 'CompanyName', 'TickSize', 'LotSize',
       'ScripCode', 'MarketLot', 'BCastFlag', 'AVMBuyMargin', 'AVMSellMargin',
       'ScripID', 'ScripName', 'GroupName', 'NdFlag', 'NDSDate', 'NDEDate',
       'SuspStatus', 'avmflag', 'SuspensionReason', 'Suspensiondate',
       'DateOfListing', 'DateOfDeListing', 'IssuePrice', 'FaceValue',
       'ISINCode', '52WeeksHigh', '52WeeksLow', 'LifeTimeHigh', 'LifeTimeLow',
       'HighDate', 'LowDate', 'MarginPercentage', 'ExchangeCode', 'SourceFile',
       'InstrumentName', 'ExpiryDate', 'StrikePrice', 'OptionType', 'CALevel',
       'PermittedToTrade', 'IssueCapital', 'WarningQty', 'FreezeQty',
       'CreditRating', 'NormalMarketStatus', 'OddLotMarketStatus',
       'SpotMarketStatus', 'AuctionMarketStatus', 'NormalMarketEligibility',
       'OddLotMarketEligibility', 'SpotMarketEligibility',
       'AuctionMarketEligibility', 'IssueRate', 'IssueStartDate',
       'InterestPaymentDate', 'IssueMaturityDate', 

In [23]:
columns_to_keep = [
    'Token', 'ShortName', 'Series','BCastFlag' ,'CompanyName', 'TickSize', 'LotSize',
    'ScripCode', 'ISINCode', 'FaceValue', 'ExchangeCode', 'InstrumentName',
    'ExpiryDate', 'StrikePrice', 'OptionType', 'MarginPercentage','SourceFile'
]


In [24]:
df_icici = merged_data_icici[columns_to_keep]
print(df_icici.head())


    Token ShortName Series BCastFlag                 CompanyName TickSize  \
0  800078    03D001     DR         0  11.10% GOI LOAN 07/04/2003    115.1   
1  800004    03E001     DR         0         GOI LOAN 5.75% 2003    110.2   
2  800032    03E002     DR         0           GOI LOAN 11% 2003   112.25   
3  800049    04C001     DR         0  12.50% GOI LOAN 23/03/2004    114.4   
4  800081    04E001     DR         0  11.50% GOI LOAN 09/05/2004    118.6   

  LotSize ScripCode      ISINCode FaceValue ExchangeCode InstrumentName  \
0       1    800078  INY019980013     100.0  CG1110S9803                  
1       1    800004  IN0019730012     100.0  CG0575S7303                  
2       1    800032  IN0019880023     100.0  CG1100S8803                  
3       1    800049  INY019930083     100.0  CG1250S9404                  
4       1    800081  INY019980054     100.0  CG1150S9804                  

  ExpiryDate StrikePrice OptionType MarginPercentage                SourceFile  
0    

In [25]:
df_icici.to_csv('icici.csv', index=False)

## kotak 

In [26]:

kotak_urls = {
    "cash": "https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_Cash_01_04_2022.txt",
    "futures": "https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_FNO_01_04_2022.txt"
}

os.makedirs('kotak', exist_ok=True)
for filename, url in kotak_urls.items():
    response = requests.get(url)
    response.raise_for_status()  
    csv_data = []
    for line in response.text.strip().splitlines():
        csv_line = [element.strip() for element in line.split('|')]
        csv_data.append(','.join(csv_line))
    csv_content = '\n'.join(csv_data)

    path = f'kotak/{filename}.csv' 
    
    with open(path, 'w') as file:
        file.write(csv_content)

In [27]:

df_cash = pd.read_csv('kotak/cash.csv', on_bad_lines='skip')
df_futures = pd.read_csv('kotak/futures.csv', on_bad_lines='skip')

df_cash = df_cash.rename(columns={'OptionType': 'optionType'})

In [28]:
df_cash.head()

,instrumentToken,instrumentName,name,lastPrice,expiry,strike,tickSize,lotSize,instrumentType,segment,exchange,isin,multiplier,exchangeToken,optionType
0,9516,20MICRONS,20 Microns Ltd,73.85,0,0,0.05,1,EQ,CASH,NSE,INE144J01027,1,16921,-
1,43865,3IINFOLTD,3i Infotech Ltd,51.25,0,0,0.05,1,EQ,CASH,NSE,INE748C01038,1,6232,-
2,891,3MINDIA,3M India Ltd,19697.30,0,0,0.05,1,EQ,CASH,NSE,INE470A01017,1,474,-
3,607,3PLAND,3P Land Holdings Ltd,14.50,0,0,0.05,1,EQ,CASH,NSE,INE105C01023,1,2595,-
4,13140,5PAISA,5Paisa Capital Ltd,344.70,0,0,0.05,1,EQ,CASH,NSE,INE618L01018,1,445,-


In [29]:
df_futures.head()

,instrumentToken,instrumentName,name,lastPrice,expiry,strike,tickSize,lotSize,instrumentType,segment,exchange,isin,multiplier,exchangeToken,optionType
0,42996,GOLD,NaN,50953.00,05APR22,0.0,1.00,1,CF,COMMODITY,MCX-CM,NaN,100,229091,XX
1,26865,GOLDM,NaN,51042.00,05APR22,0.0,1.00,100,CF,COMMODITY,MCX-CM,NaN,1,235323,XX
2,43861,CRUDEOIL,NaN,8115.00,19APR22,0.0,1.00,100,CF,COMMODITY,MCX-CM,NaN,1,234196,XX
3,163713,NATURALGAS,NaN,423.70,26APR22,0.0,0.10,1250,CF,COMMODITY,MCX-CM,NaN,1,235761,XX
4,18418,ALUMINIUM,NaN,286.65,29APR22,0.0,0.05,5,CF,COMMODITY,MCX-CM,NaN,1000,234631,XX


In [30]:
if set(df_cash.columns) == set(df_futures.columns):
    merged_data = pd.concat([df_cash, df_futures], ignore_index=True)
    print(merged_data.head()) 
else:
    print("The DataFrames do not have the same columns, cannot merge.")


   instrumentToken instrumentName                  name  lastPrice expiry  \
0             9516      20MICRONS        20 Microns Ltd      73.85      0   
1            43865      3IINFOLTD       3i Infotech Ltd      51.25      0   
2              891        3MINDIA          3M India Ltd   19697.30      0   
3              607         3PLAND  3P Land Holdings Ltd      14.50      0   
4            13140         5PAISA    5Paisa Capital Ltd     344.70      0   

   strike  tickSize  lotSize instrumentType segment exchange          isin  \
0     0.0      0.05        1             EQ    CASH      NSE  INE144J01027   
1     0.0      0.05        1             EQ    CASH      NSE  INE748C01038   
2     0.0      0.05        1             EQ    CASH      NSE  INE470A01017   
3     0.0      0.05        1             EQ    CASH      NSE  INE105C01023   
4     0.0      0.05        1             EQ    CASH      NSE  INE618L01018   

   multiplier  exchangeToken optionType  
0           1          169

In [31]:
print(df_cash.shape , df_futures.shape , merged_data.shape)

(7293, 15) (106386, 15) (113679, 15)


In [32]:
merged_data.to_csv('kotak.csv', index=False)

In [33]:
df_kotak = pd.read_csv('kotak.csv',low_memory=False)
df_kotak.head()

,instrumentToken,instrumentName,name,lastPrice,expiry,strike,tickSize,lotSize,instrumentType,segment,exchange,isin,multiplier,exchangeToken,optionType
0,9516,20MICRONS,20 Microns Ltd,73.85,0,0.0,0.05,1,EQ,CASH,NSE,INE144J01027,1,16921,-
1,43865,3IINFOLTD,3i Infotech Ltd,51.25,0,0.0,0.05,1,EQ,CASH,NSE,INE748C01038,1,6232,-
2,891,3MINDIA,3M India Ltd,19697.30,0,0.0,0.05,1,EQ,CASH,NSE,INE470A01017,1,474,-
3,607,3PLAND,3P Land Holdings Ltd,14.50,0,0.0,0.05,1,EQ,CASH,NSE,INE105C01023,1,2595,-
4,13140,5PAISA,5Paisa Capital Ltd,344.70,0,0.0,0.05,1,EQ,CASH,NSE,INE618L01018,1,445,-


## zerodha 

In [34]:

def fetch_zerodha_data():
    url_zerodha = "https://api.kite.trade/instruments"
    filename = "zerodha.csv"
    path = f"{filename}"

    print(f"Fetching data from {url_zerodha}...")

    response = requests.get(url_zerodha)
    response.raise_for_status()

    os.makedirs('datasets', exist_ok=True)
    with open(path, 'w', newline='') as file:
        writer = csv.writer(file)
        for line in response.text.splitlines():
            writer.writerow(line.split(','))
    print(f"done!")
fetch_zerodha_data()


Fetching data from https://api.kite.trade/instruments...
done!


In [35]:
zerodha_data = pd.read_csv('zerodha.csv')
zerodha_data.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,264741126,1034145,EURINR24OCTFUT,"""EURINR""",0,2024-10-29,0.0,0.0025,1,FUT,BCD-FUT,BCD
1,265969926,1038945,EURINR24NOVFUT,"""EURINR""",0,2024-11-27,0.0,0.0025,1,FUT,BCD-FUT,BCD
2,268449030,1048629,EURINR24DECFUT,"""EURINR""",0,2024-12-27,0.0,0.0025,1,FUT,BCD-FUT,BCD
3,269248262,1051751,EURINR25JANFUT,"""EURINR""",0,2025-01-29,0.0,0.0025,1,FUT,BCD-FUT,BCD
4,270018310,1054759,EURINR25FEBFUT,"""EURINR""",0,2025-02-26,0.0,0.0025,1,FUT,BCD-FUT,BCD


In [36]:
zerodha_data.columns

Index(['instrument_token', 'exchange_token', 'tradingsymbol', 'name',
       'last_price', 'expiry', 'strike', 'tick_size', 'lot_size',
       'instrument_type', 'segment', 'exchange'],
      dtype='object')

# Singular Mapping

In [37]:
fyers_df = pd.read_csv('fyers.csv',low_memory=False)  
angel_one_df = pd.read_csv('angel_one.csv',low_memory=False)  
icici_df = pd.read_csv('icici.csv',low_memory=False)  
kotak_df = pd.read_csv('kotak.csv',low_memory=False)  
zerodha_df = pd.read_csv('zerodha.csv',low_memory=False)  

In [38]:
fyers_df.rename(columns={
    'Instrument_ID': 'token',
    'Instrument_Name': 'name',
    'Lot_Size': 'lotSize',
    'Tick_Size': 'tick_size',
    'Expiry_Date': 'expiry',
    'Option_Type': 'optionType',
    'Strike_Price': 'strike'
}, inplace=True)

In [39]:
angel_one_df.rename(columns={
    'token': 'token',
    'symbol': 'tradingsymbol',
    'name': 'angel_name', 
    'expiry': 'expiry',
    'strike': 'strike',
    'lotsize': 'lotSize',
    'instrumenttype': 'optionType',
}, inplace=True)


In [40]:
icici_df.rename(columns={
    'Token': 'token',
    'ShortName': 'tradingsymbol',
    'InstrumentName': 'icici_name', 
    'ExpiryDate': 'expiry',
    'StrikePrice': 'strike',
    'LotSize': 'lotSize',
    'OptionType': 'optionType'
}, inplace=True)

In [41]:
kotak_df.rename(columns={
    'instrumentToken': 'token',
    'instrumentName': 'kotak_name', 
    'expiry': 'expiry',
    'strike': 'strike',
    'lotSize': 'lotSize',
    'optionType': 'optionType',
}, inplace=True)

In [42]:
zerodha_df.rename(columns={
    'instrument_token': 'token',
    'tradingsymbol': 'tradingsymbol',
    'name': 'zerodha_name',  
    'expiry': 'expiry',
    'strike': 'strike',
    'lot_size': 'lotSize',
    'instrument_type': 'optionType',
}, inplace=True)

In [43]:
for df in [fyers_df, angel_one_df, icici_df, kotak_df, zerodha_df]:
    df['token'] = df['token'].astype(str)  

In [44]:
merged_df = pd.merge(fyers_df, angel_one_df, on=['token', 'expiry', 'strike', 'lotSize', 'optionType'], how='outer', suffixes=('_fyers', '_angel_one'))
merged_df = pd.merge(merged_df, icici_df, on=['token', 'expiry', 'strike', 'lotSize', 'optionType'], how='outer', suffixes=('', '_icici'))
merged_df = pd.merge(merged_df, kotak_df, on=['token', 'expiry', 'strike', 'lotSize', 'optionType'], how='outer', suffixes=('', '_kotak'))
merged_df = pd.merge(merged_df, zerodha_df, on=['token', 'expiry', 'strike', 'lotSize', 'optionType'], how='outer', suffixes=('', '_zerodha'))

In [46]:
merged_df.to_csv('final_csv_data.csv', index=False)
print("Done !!!'")

Done !!!'


--> Shivam Borse